### Define Object Classes

In [10]:
class Station():

    def __init__(self,nodeId,name, sttype, mhtime):
        self.nodeId = int(nodeId)
        self.name = name
        self.sttype = sttype
        self.mhtime = mhtime
    
    def __str__(self):
        return f"Station#{self.nodeId} - {self.name} - {self.sttype}"
    
    def getType(self):
        return self.sttype
    
    def getNode(self):
        '''
        returns its node
        '''
        return self.nodeId

In [11]:
class TransportOrder():
    def __init__(self,index,source, dest, ept, ldt, cap,taskType='t'):
        self.id=index
        self.source = source
        self.dest = dest
        self.ept = ept
        self.ldt = ldt
        self.cap=cap
        self.taskType = taskType
        
        
    def __str__(self):
        return f"TO#{self.id}: From {self.source} to {self.dest}, window: {self.ept} - {self.ldt}, capability: {self.cap}"

In [12]:
class Task(TransportOrder):

    def __init__(self,name, processingTime,source, dest, ept, ldt, taskType, cap=0,index=999):
        self.name = name
        self.processingTime = processingTime
        super().__init__(index,source, dest, ept, ldt, cap,taskType)

    def __str__(self):
        return f"Task {self.name}, type:{self.taskType} - From {self.source} to {self.dest} window:{self.ept} - {self.ldt}, Minimum Processing:{self.processingTime} secs" 

In [13]:
class AGV():
    def __init__(self, agvidf, startNode, caps, speed, charge = 100, dischargeRate = 0.5, 
                 chargingRate =1, travelCost =1):
        self.chargingStations=list()
        self.agvidf = agvidf #agv id
        self.startNode = startNode #initial node
        self.caps = caps #capability
        self.speed = speed # speed in m/s
        self.charge = charge # charge %
        self.dischargeRate = dischargeRate # % per second
        self.chargingRate = chargingRate # % per second
        self.taskList = list() # list of tasks
        self.release=(0, startNode) #(time, location)
        self.travelCost = travelCost # weighted travel cost
        self.LOWER_THRESHOLD = 30 # lower threshold of charging
        self.UPPER_THRESHOLD =60 # upper threshold of charging
        self.state = 'N'
        self.lateness = 0 #calculate lateness of jobs
        self.initialSeqFile = ""
        
    def __str__(self):
        return f"AGV#{self.agvidf}, capability:{self.caps}"
    
    def addTask(self,task):
        extraCharge=0
        if self.state =='C':
            if self.release[0] < task.ept:
                extraCharge = (task.ept - self.release[0])*self.chargingRate
                self.charge = min(100,self.charge+extraCharge)
                self.state = 'N'
            else:
                task.ept = self.release[0]
                self.state = 'N'
        self.updateReleaseInformation(task,extraCharge)
        
    def getCurrentReleaseNode(self):
        return self.release[1]
    def getCurrentReleaseTime(self):
        return self.release[0]
    def setCurrentReleaseNode(self,releaseNode):
        self.release=(self.release[0],releaseNode)
        
    def updateReleaseInformation(self, task, extraCharge=0):
        print(f"AGV{self.agvidf} is at {self.release}, charge: {self.charge-extraCharge}")
        drivingDist = getDistanceFromNode(self.getCurrentReleaseNode(),task.source)+(getDistanceFromNode(task.source,task.dest))
        
        drivingTime = drivingDist/self.speed 
        
        mhtime = getStationMHT(task.source)+getStationMHT(task.dest) # get material handling time for loading and unloading
        # release time to be set to the max of ept or current release time
        self.release = ((max(task.ept,self.getCurrentReleaseTime())+(getDistanceFromNode(self.getCurrentReleaseNode(),task.source))/self.speed+
                        (getDistanceFromNode(task.source,task.dest))/self.speed)+mhtime
            ,self.release[1])
        
        self.setCurrentReleaseNode(task.dest) #agv would be at destination of the task after finishing it...
        
        self.charge = self.charge - (drivingTime * self.dischargeRate)
        print(f"{self} after finishing task:{task}, Charge:{self.charge}, release: {self.release}")
        
        #calculate lateness if any
        self.lateness+= max(0,self.getCurrentReleaseTime()-task.ldt)
        print(f'Lateness of order:{task.id},ldt:{task.ldt}, lateness:{max(0,self.getCurrentReleaseTime()-task.ldt )}, total lateness of this agv:{self.lateness}')
        
        self.taskList.append(task)
        if self.charge <=self.LOWER_THRESHOLD + 0.2*self.LOWER_THRESHOLD:
            self.createCriticalCharge()
            
    def createCriticalCharge(self):
        print(f"critical charge called at charge: {self.charge}")
        self.state ='C'
        dists = [getDistanceFromNode(self.getCurrentReleaseNode(),station.getNode()) for station in chargingStations]
        optIndex = dists.index(min(dists))
        nearestChargeNode = chargingStations[optIndex].getNode()
        
        #drive to nearest charge location and start charging for minimum required amount
        drivingDist = getDistanceFromNode(self.getCurrentReleaseNode(), nearestChargeNode)
        
        drivingTime = drivingDist/self.speed
            
        self.charge = self.charge - (drivingTime * self.dischargeRate)
        
        chargeRequired = self.LOWER_THRESHOLD - self.charge
        timeRequiredToCharge = chargeRequired / self.chargingRate
        
        task = Task("Charge", timeRequiredToCharge , self.getCurrentReleaseNode(), nearestChargeNode, 0, 0, 'C', cap=0,index=99)
        self.taskList.append(task)
        print(f"critical charge started at charge: {self.charge}")
        self.release = (self.release[0]+drivingTime+timeRequiredToCharge,self.release[1]) # point at which AGV is 30% charged
        self.setCurrentReleaseNode(nearestChargeNode) # location at which AGV is 30% charged
        self.charge = self.LOWER_THRESHOLD
        print(f"critical charge finished at: {self.release}")
    
    def getLatenessScore(self,task):
        
        lhCharge =self.charge
        lhept = max(task.ept,self.getCurrentReleaseTime())
        if self.state =='C':
            if self.release[0] < task.ept:
                extraCharge = (task.ept - self.release[0])*self.chargingRate
                lhCharge = min(100,self.charge+extraCharge)

            else:
                lhept = self.release[0]
                
        late = lhept + ((getDistanceFromNode(self.getCurrentReleaseNode(), task.source)+ \
                                                getDistanceFromNode(task.source, task.dest))/(self.speed)-task.ldt)
        
        return penalty(late)
    
    
    def getDistScore(self, task):
        return getDistanceFromNode(self.getCurrentReleaseNode(), task.source)*self.travelCost

### Penalty function for lateness

In [14]:
def penalty(x,a=1.5,b=1.1,c=-1.5):
    '''
    https://www.desmos.com/calculator/3fisjexbvp
    '''
    x=x/60   # we divide by 60 to convert time into minutes
    return max(0,(a*(pow(b,x))+c))

## Read Excel Files

In [15]:
from csv import reader

def readDistanceMatrix(distMatrixFile):
    dm = reader(open(distMatrixFile))
    global distMat #used to create global variables from a non-global scope i.e inside a function.
    distMat = list(dm)
    distMat[0][0]=0
    
    
    #print(distMat[1][4])

def getDistanceFromNode(source,dest):
    '''
    returns the distance (in m) between 2 nodes
    '''
    return float(distMat[source][dest])

#read distance matrix
readDistanceMatrix('outputDM.csv')

## Create main

In [16]:
from csv import reader
from pandas import read_excel


unscheduledTOs = list()
scheduledTOs = dict()
chargingStations = list()
stations = list()


#scheduledTOs["some"]="something"
agvs=list()

def getStationMHT(intId):
    intId = int(intId)
    st = [x for x in stations if x.nodeId == intId]
    
    if st:
        return st[0].mhtime
    

def createAGVs(agvfile):
    df =read_excel(agvfile)
    for index,row in df.iterrows():
        agv = AGV(agvidf=row['agvidf'], startNode=row['startNode'],caps= row['capability'], speed=row['speed'],
                 dischargeRate= row['dischargeRate'], chargingRate = row['chargingRate'],travelCost = row['travelCost'])
        agvs.append(agv)
        print(f'{agv} created')
    

def createRequests(demandfile):
    df =read_excel(demandfile)
    for index,row in df.iterrows():
        transportOrder = TransportOrder(row['Id'], row['source'], row['target'], row['ept'], row['ldt'], row['capability'])
        unscheduledTOs.append(transportOrder)
        print(f'{transportOrder} created')

def createStations(stationFile):
        df = read_excel(stationFile)
        for index, row in df.iterrows():
                station = Station(row['id'], row['pointidf'], row['type'], row['mhtime'])
                stations.append(station)
                if station.getType() == 'C':
                        chargingStations.append(station)
                print(f"{station} created")


def createSequenceTO():
        unscheduledTOs.sort(key = lambda x: x.ldt) # sort based on delivery time
        print("Unscheduled List Ordered based on ldt")
        numAGVs = len(agvs)
       
        for to in unscheduledTOs:
                agv_count=[agv for agv in agvs if to.cap in agv.caps]
                for c in agv_count:
                    print(f"TO#{to.id} can be done by AGV{c.agvidf} with current charge : {c.charge}%")
                

                if len(agv_count)==1:
                        agv_count[0].addTask(to)
                if len(agv_count)>1:
                        scores = []
                        
                        for agv in agv_count:
                            score = agv.getDistScore(to)+agv.getLatenessScore(to)
                            print(f"Computed score for AGV#{agv.agvidf}:{score}")
                            scores.append(score)
                            
                        optIndex = scores.index(min(scores))
                        agv_count[optIndex].addTask(to)
                
                #summarize sequence
        for agv in agvs:
            import csv
            agv.initialSeqFile = f"agv{agv.agvidf}.csv"
            with open(f"agv{agv.agvidf}.csv",'w') as f:
                f.write('Request_Number,SRC,EPT,DEST,LDT,MHT\n')
                f.write(f"{98},{agv.startNode},{0},{agv.taskList[0].source},{0},{getStationMHT(agv.startNode)}\n")
                for i,task in enumerate(agv.taskList):
                    f.write(f"{task.id},{task.source},{task.ept},{task.dest},{task.ldt},{getStationMHT(task.source)}\n")
                    if i<len(agv.taskList)-1 and agv.taskList[i+1].taskType !='C':
                        f.write(f"{100},{task.dest},{0},{agv.taskList[i+1].source},{0},{getStationMHT(task.dest)}\n")
                
                
         
            print(agv)
            for task in agv.taskList:
                    print(task)
                        
                

def solveSequenceLP():
        pass

    
# read demand file and create Transport Orders and add to scheduler list
createRequests(r'demand.xlsx') #use r to avoid errors due to / etc...

#create stations by reading excel file
createStations(r'stations.xlsx')
#create AGV objects by reading from file
createAGVs(r'agvs.xlsx')

#create initial sequence for agvs based on current demand by using clarke saving algorithm
createSequenceTO()

#Solve LP formulation to schedule tasks on agvs
solveSequenceLP()


    

TO#0: From 0 to 10, window: 30 - 90, capability: A created
TO#1: From 1 to 6, window: 30 - 75, capability: B created
TO#2: From 2 to 7, window: 60 - 75, capability: C created
TO#3: From 4 to 17, window: 90 - 120, capability: A created
TO#4: From 5 to 13, window: 105 - 150, capability: B created
TO#5: From 8 to 3, window: 120 - 280, capability: C created
TO#6: From 11 to 6, window: 150 - 201, capability: A created
TO#7: From 12 to 7, window: 240 - 430, capability: B created
TO#8: From 1 to 10, window: 300 - 447, capability: C created
TO#9: From 1 to 10, window: 320 - 506, capability: A created
TO#10: From 4 to 14, window: 350 - 461, capability: B created
TO#11: From 5 to 17, window: 400 - 481, capability: C created
TO#12: From 4 to 17, window: 425 - 547, capability: A created
TO#13: From 5 to 13, window: 440 - 600, capability: B created
TO#14: From 8 to 3, window: 491 - 554, capability: C created
TO#15: From 11 to 6, window: 524 - 623, capability: A created
TO#16: From 12 to 7, window: 

In [17]:
#KPI from initial sequence
#Calculate total lateness
tardinessGreedy = sum(l.lateness for l in agvs)

#calculate total loaded and unloaded travel

In [1]:
#Gurobi implementation
import gurobipy as grb
import pandas as pd
import numpy as np
import csv

# initialize LP model
# def solveLP(agv):

#Reading distance matrix
filename2=str("outputDM.csv")
dij = list(csv.reader(open(filename2)))
dij[0][0]=0
for i in range(len(dij)):
    for j in range(len(dij)):
        dij[i][j]=float(dij[i][j])
#Charging Nodes
C=[x.nodeId for x in chargingStations]

sp=[] #Speed of AGVs
dcr=[] #Discharging rate of AGVs
cr=[] #Charging rate of AGVs
SRC=[] #Source Nodes
EPT=[] #Earliest pick times
DEST=[] #Destination nodes
LDT=[] #Latest delivery time
MHT=[] #Material handling time at source node
REQ=[] #Request numbers

for agv in agvs:
    #Speed of AGV
    sp.append(agv.speed) #mps
    #Battery discharge rate
    dcr.append(agv.dischargeRate)
    #Battery charge rate
    cr.append(agv.chargingRate)

    #Reading requests
    req = pd.read_csv(agv.initialSeqFile)
    SRC.append(req['SRC'].tolist())
    EPT.append(req['EPT'].tolist())
    DEST.append(req['DEST'].tolist())
    LDT.append(req['LDT'].tolist())
    MHT.append(req['MHT'].tolist())
    REQ.append(req['Request_Number'].tolist())
    
#GUROBI
m = grb.Model('Scheduling')

#minimization model
m.modelSense = GRB.MINIMIZE

#decision variables
#variable for lateness of request r
Z = {(r,v):m.addVar(vtype = GRB.CONTINUOUS, lb = 0.0, name = f"Z_{r}_{v}")
     for v in range(len(agvs))  for r in range(len(REQ[v])) } # 0 becuase we do not consider earliness

#variable to represent the time instance at which AGV reaches 'source' of a request r
S = {(r,v):m.addVar(vtype = GRB.CONTINUOUS, lb = 0.0, name = f"S_{r}_{v}")
     for v in range(len(agvs))  for r in range(len(REQ[v]))}

#variable to represent the time instance at which AGV reaches 'destination' of a request r
D = {(r,v):m.addVar(vtype = GRB.CONTINUOUS, lb = 0.0, name = f"D_{r}_{v}") 
     for v in range(len(agvs))  for r in range(len(REQ[v]))}

#variable to represent the battery status at the beginning of request r
B = {(r,v):m.addVar(vtype = GRB.CONTINUOUS, lb = 30.0, ub=100.0, name = f"B_{r}_{v}")
     for v in range(len(agvs))  for r in range(len(REQ[v]))}

#battery can reach 0% only at charging stations
for v in range(len(agvs)):
    for r in range(len(REQ[v])):
        if SRC[v][r] in C:
            B[r,v].lb=0.0

#objective function
#Objective 1 - To minimize lateness
obj_lateness = grb.quicksum(Z[r,v]*Z[r,v] for r in range(len(REQ[v])) for v in range(len(agvs)))

# #Objective 2 - To maximize charging duration and minimize parking duration at source and destination nodes
obj_charging = grb.quicksum(0 if SRC[v][r] in C else 1*(S[r+1,v]-S[r,v]) for r in range(len(REQ[v])-1) for v in range(len(agvs)))

#Objective 3 - To minimize Unloaded Travel Time
obj_unloaded = grb.quicksum(0 if REQ[v][r]<98 else 1*(D[r,v] - S[r,v]) for r in range(len(REQ[v])) for v in range(len(agvs))) 
# becoz that's where unloaded travel occurs

m.setObjective(obj_lateness+obj_charging+obj_unloaded)
# 
#Constraints

#Constraint 1 - trip time should be less than Destination time instance
for v in range(len(agvs)):
    for r in range(len(REQ[v])):
        i=SRC[v][r]
        j=DEST[v][r]
        m.addConstr(S[r,v]+(MHT[v][r]+(1/sp[v])*dij[i][j])<=D[r,v],name=f"Headway_{r}_{v}")

#Constraint 2 - Destination of request should equal source of next request
for v in range(len(agvs)):
    for r in range(len(REQ[v])-1):
        i=SRC[v][r]
        j=DEST[v][r]
        m.addConstr(D[r,v]==S[r+1,v], name=f"Dest_{r}_{v}=Src{r+1}_{v}")

#Constraint 3 - A job cannot be picked up before EPT
for v in range(len(agvs)):
    for r in range(len(REQ[v])):
        m.addConstr(EPT[v][r]<=S[r,v],name=f"S_{r}_{v}>=EPT_{r}_{v}")

#Constraint 4 - To represent lateness Z = D - LDT 
for v in range(len(agvs)):
    for r in range(len(req)):
        if LDT[v][r]>0:
            m.addConstr(Z[r,v]>=D[r,v]+MHT[v][r]- LDT[v][r], name = f"Z_{r}_{v}>=D_{r}_{v}-LDT_{r}_{v}") # where Z[r] represents slack, when Z{r} is -ve, it will be 0 due to lower bound


#Constraint 5 - Battery initialization at starting nodes to 100%
for v in range(len(agvs)):
    m.addConstr(B[0,v]==100, name=f"B_0_init_{v}")

#Constraint 6 - Battery discharging and charging
for v in range(len(agvs)):
    for r in range(len(REQ[v])-1):
        i=SRC[v][r]
        j=SRC[v][r+1]
        if i in C and r >=2:
            b = m.addVar()
            m.addConstr(b==B[r,v]+((S[r+1,v]-S[r,v]-(1/sp[v])*dij[i][j])*cr[v])-((1/sp[v])*dij[i][j]*dcr[v]), name=f"b_{r}_{v}")
            m.addGenConstrMin(B[r+1,v],[b,100], name=f"B_{r}_{v}") # charge cannot be greater than 100%
        else:
            m.addConstr((B[r+1,v]==B[r,v]-((1/sp[v])*dij[i][j]*dcr[v])), name=f"B_{r}_{v}")
            
#Constraint 7 - Check for conflicts
for v1 in range(len(agvs)-1):
    for v2 in range(v1+1,len(agvs)): 
        for r1 in range(len(REQ[v1])-1):
            for r2 in range(len(REQ[v2])-1):
                if SRC[v1][r1]==SRC[v2][r2]:
                    if LDT[v1][r1]<LDT[v2][r2]:
                        m.addConstr(S[r2,v2]>=S[r1+1,v1]+1,name=f"ConflictS_v{v1}_{r1}_v{v2}_{r2}")
                    elif LDT[v1][r1]>LDT[v2][r2]:
                        m.addConstr(S[r1,v1]>=S[r2+1,v2]+1,name=f"Conflict_v{v1}_{r1}_v{v2}_{r2}")
                elif DEST[v1][r1]==DEST[v2][r2] and REQ[v1][r1]<98 and REQ[v2][r2]<98:
                    if LDT[v1][r1]<LDT[v2][r2]:
                        m.addConstr(D[r2,v2]>=D[r1+1,v1]+1,name=f"ConflictD_v{v1}_{r1}_v{v2}_{r2}")
                    elif LDT[v1][r1]>LDT[v2][r2]:
                        m.addConstr(D[r1,v1]>=D[r2+1,v2]+1,name=f"Conflict_v{v1}_{r1}_v{v2}_{r2}")
                    

#Write model to disk
m.write('model.lp')
#optimize model
m.optimize()
status = m.status
if status == GRB.Status.UNBOUNDED:
    print('The model cannot be solved because it is unbounded')
elif status == GRB.Status.OPTIMAL:
    print('The optimal objective is %g' % m.objVal)
    print(f"Variables of s:{len(S)}")
#     src = []
#     node = []
#     battery = []
#     late = sum(Z[z].x for z,y in enumerate(range(len(Z))))
#     print(f'Lateness:{late:.2f} seconds')
#     tardinessLP.append(late)
    for v in range(len(agvs)):
        for r in range(len(REQ[v])):
            print(f"{S[r,v].varName},{D[r,v].varName},{B[r,v].varName}:{S[r,v].x},{D[r,v].x},{B[r,v].x}")
#         node.append(r)
#         src.append(S[r].x)
#         battery.append(B[r].x)

    #print values
    from matplotlib import pyplot as plt
    c=['blue','coral','cyan','magenta','green','red'] 
    #Colors for Loaded Travel Time, Unloaded Travel Time, Loading Time, Unloading Time, Charging Time and Waiting Time

    plt.figure(figsize=(20,10))
    ls=['solid','dotted']
    for v in range(len(agvs)):
        bY=[] #battery level - y - axis
        bX=[] #battery time - x - axis
        for r in range(len(REQ[v])):    
            #Material Handling Time
            x1=[S[r,v].x,D[r,v].x-(1/sp[v])*dij[SRC[v][r]][DEST[v][r]]]
            y1=[SRC[v][r],SRC[v][r]]
            #Waiting Time at Source and Destination if the time exceeds 10 t.u.
            if x1[1]-x1[0]>10 and SRC[v][r] not in C:
                x1=[S[r,v].x,S[r,v].x+10]
                xw=[S[r,v].x+10,D[r,v].x-(1/sp[v])*dij[SRC[v][r]][DEST[v][r]]]
                plt.plot(xw, y1, c[5],marker='o',linestyle=ls[v],label='Waiting Time' if 'Waiting Time' 
                         not in plt.gca().get_legend_handles_labels()[1] else '')
            #Travel Time
            x2=[D[r,v].x-(1/sp[v])*dij[SRC[v][r]][DEST[v][r]],D[r,v].x]
            y2=[SRC[v][r],DEST[v][r]]
            if REQ[v][r]<98:
        #         # Original request window
        #         plt.plot([EPT[r],LDT[r]],[SRC[r]-0.5,SRC[r]-0.5],'dimgrey')
                #Loading time
                plt.plot(x1, y1, c[2],linestyle=ls[v],label='Loading Time' if 'Loading Time' 
                         not in plt.gca().get_legend_handles_labels()[1] else '')
                #Loaded travel time
                plt.plot(x2, y2, c[0],linestyle=ls[v],label='Loaded travel Time' if 'Loaded travel Time' 
                         not in plt.gca().get_legend_handles_labels()[1] else '')
            else:
                #unloaded travel time
                plt.plot(x2, y2, c[1],linestyle=ls[v],label='Unloaded travel Time' if 'Unloaded travel Time' 
                         not in plt.gca().get_legend_handles_labels()[1] else '')
                if SRC[v][r] in C:
                    #Charging time
                    plt.plot(x1, y1, c[4],linestyle=ls[v],label='Charging Time' if 'Charging Time' 
                             not in plt.gca().get_legend_handles_labels()[1] else '')
                else:
                    #Unloading time
                    plt.plot(x1, y1, c[3],linestyle=ls[v],label='Unloading Time' if 'Unloading Time' 
                             not in plt.gca().get_legend_handles_labels()[1] else '')

            #Battery status      
            bY.append(B[r,v].x)
            bX.append(S[r,v].x)
            if SRC[v][r] in C:
                bY.append(min(100,B[r,v].x+cr[v]*(D[r,v].x-S[r,v].x-(1/sp[v])*dij[SRC[v][r]][DEST[v][r]])))
            else:
                bY.append(B[r,v].x)
            bX.append(D[r,v].x-(1/sp[v])*dij[SRC[v][r]][DEST[v][r]])  

    plt.legend(loc='upper left', bbox_to_anchor=(1.05, 0.8))   
    plt.xlabel("Time (seconds)")
    plt.ylabel("Nodes")
    plt.yticks([i for i in range(20)],[i for i in range (20)])  
    plt.grid(True)
#     #Battery status representation - Uncomment the four lines below for battery status
#     plt.twinx()
#     plt.ylabel("Battery status")
#     plt.plot(bX,bY,label='Battery status',color='lightgrey')  
#     plt.yticks([20*i for i in range(1,6)])
#     plt.legend(loc='upper left', bbox_to_anchor=(1.05, 0.85))
    plt.show()
    plt.figure(figsize=(20,10))
    temp=[]
    for v in range(len(agvs)):
        temp.append(len(REQ[v]))
        from matplotlib import pyplot as plt
        c=['blue','coral','cyan','magenta','green','red'] 
        #Colors for Loaded Travel Time, Unloaded Travel Time, Loading Time, Unloading Time, Charging Time and Waiting Time
        bY=[] #battery level - y - axis
        bX=[] #battery time - x - axis        
        for r in range(len(REQ[v])):    
#             if REQ[v][r]<98:
            i=SRC[v][r]
            j=DEST[v][r]
            # Original request window
            plt.plot([EPT[v][r],LDT[v][r]-(1/sp[v])*dij[i][j]],[SRC[v][r],SRC[v][r]],'red',linestyle=ls[v],
                     marker=2,label='Request Time Window' if 'Request Time Window' 
                     not in plt.gca().get_legend_handles_labels()[1] else '')
            # AGV carries out in
            
            plt.plot([S[r,v].x, D[r,v].x-(1/sp[v])*dij[i][j]],[SRC[v][r]+0.5,SRC[v][r]+0.5],'blue',linestyle=ls[v],
                     marker=2,label='Time taken by AGV' if 'Time taken by AGV' 
                     not in plt.gca().get_legend_handles_labels()[1] else '')
    
    
#     maxticks = np.amax(temp)
#     plt.xticks([i for i in range(maxticks)],[i for i in range(maxticks)])
    plt.legend(loc='upper left', bbox_to_anchor=(1.05, 0.8))   
    plt.yticks([i for i in range(20)],[i for i in range (20)])
    plt.ylabel("Node")
    plt.xlabel("Time(Seconds)")
    plt.grid(True)
    plt.show()

#     #Add data to the agv file
#     lines=[]
#     with open(f"{agv.initialSeqFile}",'r') as f:
#         for r,line in enumerate(f):
#             if r==0:
#                 lines.append('Request_Number,SRC,EPT,DEST,LDT,MHT,Sr,Dr\n')
#                 continue
#             if r==len(list(f)):
#                 print(f"{r}<---------------------")
#                 continue
#             print('hey')
#             print(f"{line},{S[r].x},{D[r].x}")
#             lines.append(f"{line},{S[r].x},{D[r].x}\n")

#     with open('test.csv','w') as w:
#         for l in lines:
#             w.write(l)


elif status == GRB.Status.INF_OR_UNBD or status == GRB.Status.INFEASIBLE:
    print('Optimization was stopped with status %d' % status)
    # do IIS
    print('The model is infeasible; computing IIS')
    m.computeIIS()
    if m.IISMinimal:
        print('IIS is minimal\n')
    else:
        print('IIS is not minimal\n')
    print('\nThe following constraint(s) cannot be satisfied:')
    for c in m.getConstrs():
        if c.IISConstr:
            print('%s' % c.constrName)

# #     m.printAttr('X')

# tardinessLP=[]
# S=[0 for i in range(len(unscheduledTOs))]
# # for agv in agvs:
# #     solveLP(agv)
# # solveLP(agvs[0])
print("-----------------------------------KPI------------------------------------------")
print(f"Tardiness after EDD dispatching:{tardinessGreedy :.2f} seconds")
print(f"Total tardiness after solving via LP:{sum(Z[r,v].x for r in range(len(REQ[v])) for v in range(len(agvs)))} seconds")
# # solveLP(agvs[0])

FileNotFoundError: [Errno 2] No such file or directory: 'outputDM.csv'

### Improving the schedule by eliminating charge tasks that have -ve total charge effect